In [1]:
# Load the libraries with usual aliases

# Basics
import numpy as np
from numpy import inf
import pandas as pd 
import xarray as xr
import os, glob
import imageio

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib.animation as animation
from matplotlib.colors import LogNorm
from matplotlib.ticker import MultipleLocator, MaxNLocator
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Extras
import qnorm

# Global settings
plt.rcParams.update({'font.size': 20})

In [2]:
# Absolute Path
abs_path = (#"C:/Users/ennio/source/repos/erlang-data/"
            "C:/Users/ennio/source/repos/bayessmc/"
           )

# Exact directory
data_dir = "CARar_3_steps_ahead"
chart_dir = "charts"

avg_files = [ "input_avg_K0_smc_grid_21x21_T_144"
            , "r_p1_avg_K500_smc_grid_21x21_T_144"
            , "r_p2_avg_K500_smc_grid_21x21_T_144"
            , "r_p3_avg_K500_smc_grid_21x21_T_144" ]



var_files = [ "input_var_K500_smc_grid_21x21_T_144"
            , "r_p1_var_K500_smc_grid_21x21_T_144"
            , "r_p2_avg_K500_smc_grid_21x21_T_144"
            , "r_p3_var_K500_smc_grid_21x21_T_144" ]

interesting_snaps = [1, 49, 111] # T = 0:00, 8:00, 19:10

In [3]:
# Plotting the graph 
# Read data from file 'path + directory/filename.csv' 
def plot_graph(filename, directory=data_dir, path=abs_path, index="", func=lambda x: x):
    data = pd.read_csv(path + directory + "/" + filename + ".csv")
    if index != "":
        data = data.set_index(index)
    data = func(data)
    fig = plt.figure()
    fig.suptitle(filename)
    fig.set_size_inches(20,10)
    plt.imshow(data.values, aspect='auto')
    plt.colorbar()
    plt.show()

In [4]:
# Plotting a multi-graph based on a list of `filenames` 
# Reads the data from file 'path + directory/filenames[i].csv' 
# executes func(files) as a pre-processing step
def plot_multi(filenames, func=lambda x: x, directory=data_dir, path=abs_path, maxcols=2, width=20, height=10):
    # Data Preparation
    nrows = - (- len(filenames) // maxcols)
    d = list(map(lambda x: pd.read_csv(path + directory + "/" + x + ".csv").set_index("Space"), filenames))
    d = list(map(func, d))
    dmin = min(map(lambda y: y.min().min(), d))
    dmax = max(map(lambda x: x.max().max(), d))
    
    # Figure Generation
    plt.figure(1)
    fig, axes = plt.subplots(nrows, maxcols)
    i = 0
    for data, ax in zip(d, axes.ravel()):
        ax.set_title(filenames[i])
        im = ax.imshow(data.values, aspect='auto', vmin=dmin, vmax=dmax)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="20%", pad=0.05)
        cbar = plt.colorbar(im, cax=cax, format="%.2f")
        i = i + 1
    fig.set_size_inches(width, height)
    plt.show()

In [5]:
def generate_gif(filename, path=abs_path, input_dir=data_dir, directory=chart_dir):
    df = pd.read_csv(path + input_dir + "/" + filename + ".csv")
    
    dmin = df.drop("Space", axis=1).replace([np.inf, -np.inf], 0).to_numpy().min()
    dmax = df.drop("Space", axis=1).replace([np.inf, -np.inf], 0).to_numpy().max()
    
    # Data refactoring
    data = []
    for j in range(1, 141):
        dfs = []
        i = 0
        for i in range(0, 21): 
            dfs.append(df.iloc[(21 * i):(21 * i + 21), j:(j + 1)].to_numpy())

        dfs = np.asarray(dfs)
        data.append(np.reshape(dfs, (21, 21)))
        
    with plt.rc_context({'font.size': 16}):
        # Plots with Fixed colorbar    
        for i in np.arange(140):
            plt.figure()
            plt.title(filename + '\nTime point: ' + str(i), pad="10")
            plt.imshow(data[i], origin='lower', vmin=dmin, vmax=dmax), plt.colorbar()
            plt.savefig(path + directory + '/temp/' + 'snap_' + str(i) + '_' + filename + '.png')

    snap1 = glob.glob(path + directory + '/temp/' + 'snap_[0-9]_' + filename + '.png') 
    snap2 = glob.glob(path + directory + '/temp/' + 'snap_[0-9][0-9]_' + filename + '.png')
    snap3 = glob.glob(path + directory + '/temp/' + 'snap_[0-9][0-9][0-9]_' + filename + '.png')
    snaps = snap1 + snap2 + snap3

    fix = [imageio.imread(file) for file in snaps]

    imageio.mimsave(path + directory + '/movie_' + filename + '.gif', fix, fps = 3)

In [6]:
def multi_snap_printer(filename, snaps, directory=data_dir, path=abs_path, maxcols=3, width=0.05, left=0.85
                      ,smin=0, smax=0, cmap='', symmetric=False):
    df = pd.read_csv(path + directory + '/' + filename + ".csv")

    # Data refactoring
    d = []
    for j in snaps:
        dfs = []
        i = 0
        for i in range(0, 21): 
            dfs.append(df.iloc[(21 * i):(21 * i + 21), j:(j + 1)].to_numpy())

        dfs = np.asarray(dfs)
        d.append(np.reshape(dfs, (21, 21)))
    
    d2 = np.nan_to_num(d, neginf=0, posinf=0)
    dmax = max(list(map(lambda x: x.max(), d2)))
    dmin = max(list(map(lambda x: x.min(), d2)))
    if(abs(dmax) > abs(dmin) and symmetric):
        dmin = - dmax
    elif(symmetric):
        dmax = - dmin
    #print("MAX:" + str(dmax) + "| MIN:" + str(dmin))
    if smin != 0:
        dmin = smin
        d = np.nan_to_num(d, neginf=smin)
    if smax != 0:
        dmax = smax
        d = np.nan_to_num(d, posinf=smin)
    
    # Grid Ticks
    major_ticks = np.arange(4.5, 20, 5)
    minor_ticks = np.arange(0.5, 20, 1)

    # Figure
    plt.figure(1)
    if maxcols != 1:
        fig, axes = plt.subplots(1, maxcols)
        i = 0
        for data, ax in zip(d, axes.ravel()):
            ax = _helper(data, snaps, ax, i, fig, dmin, dmax, cmap, major_ticks, minor_ticks, width, left)
            fig.set_size_inches(20, 5)
            i = i + 1
    else: 
        fig, axes = plt.subplots(1, 1)
        axes = _helper(d[0], snaps, axes, 0, fig, dmin, dmax, cmap, major_ticks, minor_ticks, width, left)
        fig.set_size_inches(6, 5)
    
    plt.show()
    
def _helper(data, snaps, ax, n, fig, dmin, dmax, cmap, major_ticks, minor_ticks, width, left):
    hour = (snaps[n] - 1) // 6
    minutes = ((snaps[n] - 1) % 6) * 10
    if minutes == 0:
        minutes = "00"
    #ax.set_title("snap_t" + str(snaps[i]-1) + '_' + filename)
    if cmap == '':
        im = ax.imshow(data, origin='lower', aspect='auto', vmin=dmin, vmax=dmax)
    else:
        im = ax.imshow(data, origin='lower', aspect='auto', vmin=dmin, vmax=dmax, cmap=cmap)

    ax.set_title("Clock Time: " + str(hour) + ":" + str(minutes), pad="10")
    ax.set_xticks(major_ticks)
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)
    ax.set_ylim(bottom=-0.5)
    ax.set_xlim(left=-0.5)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(which='major', color='#000000', linestyle='--')
    ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([left, 0.15, width, 0.7])
    fig.colorbar(im, cax=cbar_ax)
    
    return ax

In [7]:
# Input Normalization (Enable when needed)
'''
# Input Normalization 
base = "_trajectories_grid_21x21_T_144" 
files = []
for i in range(1, 101):
    if i < 10:
        s = "00" + str(i)
    elif i < 100:
        s = "0" + str(i)
    else:
        s = str(i)
    files.append(s + base)

for f in files:
    df = pd.read_csv(path + directory + "/" + f + ".csv")
    df = qnorm.quantile_normalize(df)
    #df.drop(columns="", axis=1, inplace=True)
    df.to_csv(path + directory + "/norm/" + f + ".csv", index=False)
'''

'\n# Input Normalization \nbase = "_trajectories_grid_21x21_T_144" \nfiles = []\nfor i in range(1, 101):\n    if i < 10:\n        s = "00" + str(i)\n    elif i < 100:\n        s = "0" + str(i)\n    else:\n        s = str(i)\n    files.append(s + base)\n\nfor f in files:\n    df = pd.read_csv(path + directory + "/" + f + ".csv")\n    df = qnorm.quantile_normalize(df)\n    #df.drop(columns="", axis=1, inplace=True)\n    df.to_csv(path + directory + "/norm/" + f + ".csv", index=False)\n'

In [8]:
# Average & Variance Plotting
plot_multi(avg_files, lambda x: x, maxcols=4)
plot_multi(var_files, lambda x: x, maxcols=4)

input_trajectory = "data_matrix_20131111"

plot_graph(input_trajectory, func=lambda x: x)
plot_graph(input_trajectory, func=qnorm.quantile_normalize)
plot_graph(input_trajectory, func=lambda x: qnorm.quantile_normalize(x.transpose()).transpose())

filename = "r_p3_avg_K500_smc_grid_21x21_T_144"

plot_graph(filename, index="Space", func=qnorm.quantile_normalize)
#plot_graph(filename, index="Space", func=lambda x: qnorm.quantile_normalize(x.transpose()).transpose())

# Normalized Input
#plot_multi(avg_files, directory=data_dir + "/norm", maxcols=4)
#plot_multi(var_files, directory=data_dir + "/norm", maxcols=4)

# Normalized Output
#plot_multi(avg_files, func=qnorm.quantile_normalize)
#plot_multi(var_files, func=qnorm.quantile_normalize)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/ennio/source/repos/bayessmc/CARar_3_steps_ahead/input_avg_K0_smc_grid_21x21_T_144.csv'

In [ ]:
# Property P1 plotting..
# plot_graph("r_p1_avg_K500_smc_grid_21x21_T_144", index="Space")

In [ ]:
# INPUT GIF
filename = "input_avg_K0_smc_grid_21x21_T_144"

#generate_gif(filename)

# P1 GIF
prop = 'p1'
filename = "r_" + prop + "_avg_K500_smc_grid_21x21_T_144"

#generate_gif(filename)

# P2' GIF
prop = 'p2'
filename = "r_" + prop + "_avg_K500_smc_grid_21x21_T_144"

#generate_gif(filename)

# P3 GIF
prop = 'p3'
filename = "r_" + prop + "_avg_K500_smc_grid_21x21_T_144"

#generate_gif(filename)

In [ ]:
multi_snap_printer(avg_files[0], interesting_snaps, width=0.040, left=0.84)
multi_snap_printer(avg_files[1], interesting_snaps, width=0.040, left=0.84, cmap='PuOr', symmetric=True)
multi_snap_printer(avg_files[2], interesting_snaps, width=0.040, left=0.84, cmap='PuOr', symmetric=True)
multi_snap_printer(avg_files[3], interesting_snaps, width=0.040, left=0.84, cmap='PuOr', symmetric=True)

In [ ]:
s_avg_files = list(map(lambda x: x.replace("r_", "s_"), avg_files))

#multi_snap_printer(s_avg_files[0], interesting_snaps, width=0.040, left=0.84)
multi_snap_printer(s_avg_files[1], interesting_snaps, width=0.040, left=0.84, smax=1.0, smin=0.01, cmap='RdYlGn')
multi_snap_printer(s_avg_files[2], interesting_snaps, width=0.040, left=0.84, smax=1.0, smin=0.01, cmap='RdYlGn')
multi_snap_printer(s_avg_files[3], interesting_snaps, width=0.040, left=0.84, smax=1.0, smin=0.01, cmap='RdYlGn')

In [ ]:
prop = 'poi'
files = ['g6_r_' + prop + "_avg_K500_smc_grid_21x21_T_144", 'g6_s_' + prop + "_avg_K500_smc_grid_21x21_T_144"]

multi_snap_printer(files[0], interesting_snaps, width=0.040, left=0.84, cmap='PuOr', symmetric=True, smin=-400, smax=400)
multi_snap_printer(files[1], interesting_snaps, width=0.040, left=0.84, cmap='RdYlGn', smax=1.0, smin=0.01)

#for f in files:
    #generate_gif(f)

In [ ]:
s_p11 = ["s_p11_avg_K500_smc_grid_21x21_T_144", "s_p11_var_K500_smc_grid_21x21_T_144"] 
r_p11 = ["r_p11_avg_K500_smc_grid_21x21_T_144", "r_p11_var_K500_smc_grid_21x21_T_144"]

plot_multi(s_p11)
plot_multi(r_p11)

In [ ]:
multi_snap_printer(s_p11[0], [1], maxcols=1, smax=1.0, smin=0.01, cmap='RdYlGn')
multi_snap_printer(r_p11[0], [1], maxcols=1, cmap='PuOr', symmetric=True)

In [ ]:
prop = 'poi'
files = ['r_' + prop + "_avg_K500_smc_grid_21x21_T_144", 's_' + prop + "_avg_K500_smc_grid_21x21_T_144"]

for f in files:
    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)

In [ ]:
prop = 'p33'

filename = 'r_' + prop + "_avg_K500_smc_grid_21x21_T_144"
#generate_gif(filename)

filename = 's_' + prop + "_avg_K500_smc_grid_21x21_T_144"
#generate_gif(filename)

In [ ]:
prop = 'poi'
#files = ['g4_r_' + prop + "_avg_K500_smc_grid_21x21_T_144", 's_' + prop + "_avg_K500_smc_grid_21x21_T_144"]

#for f in files:
#    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)
    
files = ['g5_r_' + prop + "_avg_K500_smc_grid_21x21_T_144", 'g5_s_' + prop + "_avg_K500_smc_grid_21x21_T_144"]

for f in files:
    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)

In [ ]:
files = ["r_p1_avg_K500_smc_grid_21x21_T_144", "s_p1_avg_K500_smc_grid_21x21_T_144"]

for f in files:
    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)

In [9]:
files = ["r_p3_avg_K500_smc_grid_21x21_T_144", "s_p3_avg_K500_smc_grid_21x21_T_144"]

for f in files:
    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/ennio/source/repos/bayessmc/CARar_3_steps_ahead/r_p3_avg_K500_smc_grid_21x21_T_144.csv'

In [ ]:
files = ["r_p2_avg_K500_smc_grid_21x21_T_144", "s_p2_avg_K500_smc_grid_21x21_T_144"]

for f in files:
    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)

In [ ]:
files = ["r_p4_avg_K500_smc_grid_21x21_T_144", "s_p4_avg_K500_smc_grid_21x21_T_144"]

for f in files:
    multi_snap_printer(f, interesting_snaps, width=0.040, left=0.84)